# CVXPy Mixed-Integer Quadratic Programming

This notebook solves the portfolio optimization problem using **CVXPy**, a Python library for convex optimization that allows us to formulate the problem as a **Mixed-Integer Quadratic Program (MIQP)**.

## Problem Formulation

$$\begin{align}
\text{minimize} \quad & x^T Q x + q^T x \\
\text{subject to} \quad & \sum_{i=1}^n x_i = B \\
& x_i \in \{0, 1\} \quad \forall i
\end{align}$$

where:
- $x \in \{0,1\}^n$: Binary selection vector
- $Q \in \mathbb{R}^{n \times n}$: Quadratic cost matrix (scaled covariance)
- $q \in \mathbb{R}^n$: Linear cost vector (scaled returns)
- $B$: Cardinality constraint (number of assets)

## Solver: ECOS_BB

**ECOS_BB** (Embedded Conic Solver with Branch-and-Bound) is a free, open-source solver for mixed-integer problems that comes pre-installed with CVXPy.

### Advantages
- ✅ **Free and open-source**: No license required
- ✅ Pre-installed with CVXPy
- ✅ Efficient for small-to-medium problems (n < 50)

### Limitations
- ❌ **NP-Hard complexity**: Worst-case exponential time
- ❌ May be slower than commercial solvers (Gurobi, CPLEX)

---


In [1]:
import cvxpy as cp
import numpy as np
import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')

## 1. Load QUBO Data

Load the problem data generated in `Data-and-QUBO.ipynb`.

In [2]:
# Load QUBO problem data
data = np.load("portfolio_qubo_data.npz")

Q = data['Q']
q = data['q']
mu = data['mu']
Sigma = data['Sigma']
B = int(data['B'])
TICKERS = list(data['TICKERS'])
n = len(TICKERS)

print("✓ Data loaded successfully.")
print(f"  n = {n} assets")
print(f"  B = {B} cardinality")
print(f"  Q shape: {Q.shape}")
print(f"  q shape: {q.shape}")

✓ Data loaded successfully.
  n = 21 assets
  B = 4 cardinality
  Q shape: (21, 21)
  q shape: (21,)


## 2. Solve Optimization Problem with CVXPy

CVXPy allows us to express the optimization problem in a natural mathematical syntax and automatically handles the mixed-integer structure.


The solver automatically handles the mixed-integer structure.


In [3]:
print("\n" + "="*60)
print("           CVXPy MIQP OPTIMIZATION")
print("="*60)

# Define optimization variables
x = cp.Variable(n, boolean=True)

# Define objective function: minimize x^T Q x + q^T x
objective = cp.Minimize(cp.quad_form(x, Q) + q @ x)

# Define constraints: sum(x) == B
constraints = [cp.sum(x) == B]

# Create optimization problem
problem = cp.Problem(objective, constraints)

# Solve with ECOS_BB (free solver included with CVXPy)
print("\n🔧 Solver: ECOS_BB (Branch-and-Bound)")
print("   Method: Mixed-integer conic programming")
print(f"   Problem size: {n} binary variables, 1 constraint\n")

t_start = time.perf_counter()

try:
    problem.solve(solver=cp.ECOS_BB, verbose=True)
    t_end = time.perf_counter()
    t_cvxpy = t_end - t_start
    
    # Check solver status
    if problem.status not in ["optimal", "optimal_inaccurate"]:
        raise ValueError(f"Solver failed with status: {problem.status}")
    
    print(f"\n✓ Optimization completed successfully")
    print(f"   Status: {problem.status}")
    print(f"   Solve time: {t_cvxpy:.4f} seconds")
    
except cp.error.SolverError as e:
    print(f"\n✗ Solver error: {e}")
    raise

except Exception as e:
    print(f"\n✗ Unexpected error: {e}")
    raise

(CVXPY) Nov 22 10:52:23 PM: Your problem has 21 variables, 1 constraints, and 0 parameters.
(CVXPY) Nov 22 10:52:23 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 22 10:52:23 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 22 10:52:23 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Nov 22 10:52:23 PM: Your problem is compiled with the CPP canonicalization backend.
(CVXPY) Nov 22 10:52:23 PM: Compiling problem (target solver=ECOS_BB).
(CVXPY) Nov 22 10:52:23 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> ECOS_BB
(CVXPY) Nov 22 10:52:23 PM: Applying reduction Dcp2Cone
(CVXPY) Nov 22 10:52:23 PM: Applying reduction CvxAttr2Constr
(CVXPY) Nov 22 10:52:23 PM: Applying reduction ConeMatrixStuffing
(CVXPY) Nov 22 10:52:23 PM: Applying reduction ECOS_BB
(CVXPY) Nov 22 10:52:23 PM: Finished problem compilation 


           CVXPy MIQP OPTIMIZATION

🔧 Solver: ECOS_BB (Branch-and-Bound)
   Method: Mixed-integer conic programming
   Problem size: 21 binary variables, 1 constraint

                                     CVXPY                                     
                                     v1.7.3                                    
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
-------------------------------------------------------------------------------
                                    Summary                                    
-------------------------------

## 3. Extract and Validate Solution

Extract the optimal binary vector and validate that it satisfies the cardinality constraint.

In [4]:
# Extract solution
x_opt = x.value
x_bin = np.round(x_opt).astype(int)  # Round to handle numerical precision

# Validate solution
assert np.sum(x_bin) == B, f"Invalid solution: sum(x) = {np.sum(x_bin)} != {B}"
assert np.all((x_bin == 0) | (x_bin == 1)), "Solution must be binary"

print("\n✓ Solution validation passed")
print(f"   Binary vector: {x_bin}")
print(f"   Cardinality: {np.sum(x_bin)} (expected: {B})")


✓ Solution validation passed
   Binary vector: [0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1]
   Cardinality: 4 (expected: 4)


## 4. Calculate Financial Metrics

Compute annualized return, volatility, and Sharpe ratio for the optimal portfolio.

In [5]:
# Calculate QUBO cost
fx_cvx = float(x_bin @ Q @ x_bin + q @ x_bin)

# Calculate financial metrics
sel_idx_CVX = np.where(x_bin == 1)[0]
selected_tickers = [TICKERS[i] for i in sel_idx_CVX]

w = np.zeros(n)
w[sel_idx_CVX] = 1.0 / B  # Equal-weighted portfolio

mu_day = float(mu @ w)
var_day = float(w @ Sigma @ w)

mu_ann_CVX = 252 * mu_day
std_ann_CVX = np.sqrt(252 * var_day)
sharpe_CVX = mu_ann_CVX / std_ann_CVX

print("\n" + "="*60)
print("              OPTIMIZATION RESULTS")
print("="*60)
print(f"\n📊 Objective Function:")
print(f"   f(x) = {fx_cvx:.6f}")

print(f"\n💼 Optimal Portfolio (B={B} assets):")
for i, ticker in enumerate(selected_tickers, 1):
    weight_pct = (1/B) * 100
    print(f"   {i}. {ticker:6s} - {weight_pct:.2f}%")

print(f"\n📈 Financial Metrics:")
print(f"   Annualized Return:     μ = {mu_ann_CVX:.6f} ({mu_ann_CVX*100:.2f}%)")
print(f"   Annualized Volatility: σ = {std_ann_CVX:.6f} ({std_ann_CVX*100:.2f}%)")
print(f"   Sharpe Ratio:         SR = {sharpe_CVX:.3f}")

print(f"\n⏱️  Computation Time: {t_cvxpy:.4f} seconds")
print("="*60)


              OPTIMIZATION RESULTS

📊 Objective Function:
   f(x) = -0.001198

💼 Optimal Portfolio (B=4 assets):
   1. MSFT   - 25.00%
   2. JNJ    - 25.00%
   3. JPM    - 25.00%
   4. MS     - 25.00%

📈 Financial Metrics:
   Annualized Return:     μ = 0.477126 (47.71%)
   Annualized Volatility: σ = 0.187264 (18.73%)
   Sharpe Ratio:         SR = 2.548

⏱️  Computation Time: 0.1265 seconds
